# Deep Agent Memory Management

Manage long-term memories for your LangGraph agents. Changes here sync with LangSmith Studio.

**Setup**: Run `langgraph dev` from `deep-agent/` first.

**Graphs**: `main-agent`, `analysis-agent`, `web-research-agent`, `credibility-agent`

In [11]:
import json
from datetime import datetime
from langgraph_sdk import get_sync_client

LANGGRAPH_URL = "http://localhost:2024"

try:
    client = get_sync_client(url=LANGGRAPH_URL)
    all_assistants = client.assistants.search(limit=100)
    print(f"Connected to LangGraph server at {LANGGRAPH_URL}")
except Exception as e:
    print(f"Could not connect: {e}\nRun 'langgraph dev' from the deep-agent/ directory first.")
    raise SystemExit(1)

# Discover graphs from existing assistants
AVAILABLE_GRAPHS = list(set(a["graph_id"] for a in all_assistants if a.get("graph_id")))
if not AVAILABLE_GRAPHS:
    AVAILABLE_GRAPHS = ["main-agent", "analysis-agent", "web-research-agent", "credibility-agent"]
print(f"Available graphs: {AVAILABLE_GRAPHS}")

Connected to LangGraph server at http://localhost:2024
Available graphs: ['analysis-agent', 'credibility-agent', 'web-research-agent', 'main-agent']


## How It Works

Each graph gets a default assistant from `langgraph dev`. We find those assistants and use their namespaces `[assistant_id, "filesystem"]` to read/write memories. Same IDs as LangSmith Studio.

In [12]:
# Cache assistants and build namespace lookup
ASSISTANTS = {}
for graph in AVAILABLE_GRAPHS:
    assistants = client.assistants.search(graph_id=graph, limit=1)
    if assistants:
        ASSISTANTS[graph] = assistants[0]
        print(f"{graph}: {assistants[0]['assistant_id']}")

def get_namespace(graph: str) -> tuple:
    """Get the memory namespace for a graph."""
    if graph not in ASSISTANTS:
        raise ValueError(f"No assistant for '{graph}'. Run it in LangSmith Studio first.")
    return (ASSISTANTS[graph]["assistant_id"], "filesystem")

def normalize_key(key: str) -> str:
    """Ensure key starts with /"""
    return key if key.startswith("/") else f"/{key}"

def format_content(value: dict) -> str:
    """Format memory content for display."""
    if isinstance(value.get("content"), list):
        return "\n".join(value["content"])
    return json.dumps(value, indent=2)

analysis-agent: 854aa6d0-4ddb-5980-837f-8db6045be50f
credibility-agent: 91e1bc6c-3ce8-58e1-9506-d229476e35f8
web-research-agent: 3e46236d-ede5-5fd4-91d9-6fd4977ad898
main-agent: 9cbab8c9-1a30-54df-a839-d7e26aca6464


## View Memories

List all memories for a graph or get a specific file.

In [13]:
def list_memories(graph: str = "main-agent"):
    """List all memories for a graph."""
    items = client.store.search_items(get_namespace(graph), limit=100)["items"]
    
    print(f"\n=== {graph} ({len(items)} memories) ===\n")
    for item in items:
        content = format_content(item["value"])
        preview = content[:300] + "..." if len(content) > 300 else content
        print(f"[{item['key']}]\n{preview}\n")
    return items

# Example
list_memories("main-agent")


=== main-agent (2 memories) ===

[/source_notes.txt]
## Source Notes
- Treat ts2.tech as a secondary/tertiary commentary site; use it only for context and always verify factual assertions (dates, quantities, prices) against primary filings or Tier-1 newswires.


[/website_quality.txt]
## Website Quality Notes
- Yahoo Finance press-release pages are generally reliable mirrors of company/PR wire announcements but should be cross-checked with original RNS/Investegate or company site when confirming precise deal terms.




[{'namespace': ['9cbab8c9-1a30-54df-a839-d7e26aca6464', 'filesystem'],
  'key': '/source_notes.txt',
  'value': {'content': ['## Source Notes',
    '- Treat ts2.tech as a secondary/tertiary commentary site; use it only for context and always verify factual assertions (dates, quantities, prices) against primary filings or Tier-1 newswires.',
    ''],
   'created_at': '2025-12-24T10:51:16.945366+00:00',
   'modified_at': '2025-12-24T10:51:16.945366+00:00'},
  'created_at': '2025-12-24T10:51:16.945559+00:00',
  'updated_at': '2025-12-24T10:51:16.945563+00:00',
  'score': None},
 {'namespace': ['9cbab8c9-1a30-54df-a839-d7e26aca6464', 'filesystem'],
  'key': '/website_quality.txt',
  'value': {'content': ['## Website Quality Notes',
    '- Yahoo Finance press-release pages are generally reliable mirrors of company/PR wire announcements but should be cross-checked with original RNS/Investegate or company site when confirming precise deal terms.',
    ''],
   'created_at': '2025-12-24T10:51:1

In [14]:
def get_memory(key: str, graph: str = "main-agent"):
    """Get a specific memory by key."""
    item = client.store.get_item(get_namespace(graph), normalize_key(key))
    if item:
        print(f"[{graph}] {key}\n{format_content(item['value'])}")
        return item
    print(f"Not found: {key}")
    return None

# Example
get_memory("website_quality.txt", "main-agent")

[main-agent] website_quality.txt
## Website Quality Notes
- Yahoo Finance press-release pages are generally reliable mirrors of company/PR wire announcements but should be cross-checked with original RNS/Investegate or company site when confirming precise deal terms.



{'namespace': ['9cbab8c9-1a30-54df-a839-d7e26aca6464', 'filesystem'],
 'key': '/website_quality.txt',
 'value': {'content': ['## Website Quality Notes',
   '- Yahoo Finance press-release pages are generally reliable mirrors of company/PR wire announcements but should be cross-checked with original RNS/Investegate or company site when confirming precise deal terms.',
   ''],
  'created_at': '2025-12-24T10:51:16.945482+00:00',
  'modified_at': '2025-12-24T10:51:16.945482+00:00'},
 'created_at': '2025-12-24T10:51:16.945767+00:00',
 'updated_at': '2025-12-24T10:51:16.945768+00:00'}

## Save & Update Memories

Create new memories, overwrite existing ones, append content, or seed defaults.

In [15]:
def save_memory(key: str, content: str, graph: str = "main-agent"):
    """Save or update a memory."""
    key = normalize_key(key)
    ts = datetime.now().isoformat()
    client.store.put_item(
        get_namespace(graph), key,
        value={"content": content.split("\n"), "created_at": ts, "modified_at": ts}
    )
    print(f"Saved: {key} ({graph})")

def append_memory(key: str, content: str, graph: str = "main-agent"):
    """Append content to an existing memory."""
    key = normalize_key(key)
    existing = client.store.get_item(get_namespace(graph), key)
    if existing and "content" in existing["value"]:
        current = "\n".join(existing["value"]["content"])
        content = current + "\n\n" + content
    save_memory(key, content, graph)

SEED_MEMORIES = {
    "website_quality.txt": """# Website Quality Ratings

## High Quality
- reuters.com (5/5) - Timestamped headlines, strong for M&A/earnings
- sec.gov/edgar (5/5) - Primary filings, reliable timestamps
- bls.gov / bea.gov (5/5) - Official macro data

## Lower Quality  
- seekingalpha.com (2/5) - Opinion-heavy, missing timestamps
- twitter.com/x.com (2/5) - Sentiment only, not factual""",

    "research_lessons.txt": """# Research Lessons

## Effective Patterns
- "TICKER earnings 8-K" for primary documents
- site:reuters.com TICKER for timestamped headlines
- Cross-check headline vs filing time for price moves

## What Didn't Work
- Generic "company news" returns SEO blogs
- Social clips rarely cite sources""",

    "source_notes.txt": """# Source Notes

## reuters.com - Reliable datelines, good for double-sourcing
## sec.gov/edgar - 8-K for news, 10-Q/10-K for numbers  
## ft.com - Strong regulatory context, paywalled""",

    "coding.txt": """# Coding Lessons

- Normalize timestamps to UTC, convert to ET for market context
- Log dataframe shapes after merges
- Close matplotlib figures after saving"""
}

def seed_memories(graph: str = "main-agent"):
    """Seed default memories for a graph."""
    for key, content in SEED_MEMORIES.items():
        save_memory(key, content, graph)
    print(f"Seeded {len(SEED_MEMORIES)} memories for '{graph}'")

In [ ]:
# Seed all graphs: 
# for graph in AVAILABLE_GRAPHS: seed_memories(graph)

Saved: /website_quality.txt (analysis-agent)
Saved: /research_lessons.txt (analysis-agent)
Saved: /source_notes.txt (analysis-agent)
Saved: /coding.txt (analysis-agent)
Seeded 4 memories for 'analysis-agent'
Saved: /website_quality.txt (credibility-agent)
Saved: /research_lessons.txt (credibility-agent)
Saved: /source_notes.txt (credibility-agent)
Saved: /coding.txt (credibility-agent)
Seeded 4 memories for 'credibility-agent'
Saved: /website_quality.txt (web-research-agent)
Saved: /research_lessons.txt (web-research-agent)
Saved: /source_notes.txt (web-research-agent)
Saved: /coding.txt (web-research-agent)
Seeded 4 memories for 'web-research-agent'
Saved: /website_quality.txt (main-agent)
Saved: /research_lessons.txt (main-agent)
Saved: /source_notes.txt (main-agent)
Saved: /coding.txt (main-agent)
Seeded 4 memories for 'main-agent'


## Delete Memories

Remove individual memories or clear all for a graph.

In [ ]:
def delete_memory(key: str, graph: str = "main-agent"):
    """Delete a specific memory."""
    client.store.delete_item(get_namespace(graph), normalize_key(key))
    print(f"Deleted: {key} ({graph})")

def clear_memories(graph: str = "main-agent"):
    """Delete ALL memories for a graph."""
    ns = get_namespace(graph)
    items = client.store.search_items(ns, limit=100)["items"]
    for item in items:
        client.store.delete_item(ns, item["key"])
    print(f"Cleared {len(items)} memories ({graph})")

## Quick Reference

```python
AVAILABLE_GRAPHS  # ["main-agent", "analysis-agent", "web-research-agent", "credibility-agent"]

list_memories("main-agent")                          # List all memories
get_memory("website_quality.txt", "main-agent")      # Get one memory

save_memory("file.txt", "content", "main-agent")     # Save/overwrite
append_memory("file.txt", "more", "main-agent")      # Append
seed_memories("main-agent")                          # Seed defaults

delete_memory("file.txt", "main-agent")              # Delete one
clear_memories("main-agent")                         # Clear graph
```